In [2]:
! pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 2.0 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 128.1 MB/s eta 0:00:01
     |████████████████████████████████| 184 kB 144.4 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 83.8 MB/s  eta 0:00:01
     |████████████████████████████████| 140 kB 132.0 MB/s eta 0:00:01
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=48895f22d141d0672a0cd949d05bcd2eb979736089b4d4eddd5e6c8674f436fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-7qc7c95g/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9


In [1]:
import torch
import torch.nn as nn
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments,AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import time

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.8/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [2]:
path= '/workspace/deerproject2/'

In [3]:
device='cuda:0'

In [5]:
train_data = pd.read_csv(path+ 'new_train.csv')

test_data = pd.read_csv(path+ 'new_test.csv')


In [6]:
train_data.head(5)

,Unnamed: 0,Class Index,Title,Description
0,51877,2,"When it comes to the playoffs, the more the me...","Of all the changes in sports, the one that fun..."
1,53442,4,Flip Open That Cellphone: It's IM on the Move,"Although it grew up on desktop PCs, instant-me..."
2,102638,4,Video watchdogs say almost all new games are M...,This season #39;s blockbuster video games give...
3,76280,1,Militants slaughter 11 Iraqi soldiers abducted...,"Insurgents slaughtered 11 Iraqi soldiers, behe..."
4,28453,2,"As an NHL lockout looms, Canada cooly outduels...","TORONTO Midway through the third period, with ..."


In [7]:
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=4)
# Load the Longformer tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expec

In [8]:
train_label = []
train_text = []
train_title = []
for _,row in train_data.iterrows():
  train_label.append(row['Class Index']-1)
  train_title.append(row['Title'])
  train_text.append({'text': row['Description']})

In [9]:
test_label = []
test_text = []
test_title = []
for _,row in test_data.iterrows():
  test_label.append(row['Class Index']-1)
  test_title.append(row['Title'])
  test_text.append({'text': row['Description']})

In [10]:
len(train_text)

6000

In [11]:
len(test_text)

1520

In [12]:
for i in range(len(train_label)):
    train_text[i]['label'] = train_label[i]

In [13]:
for i in range(len(test_label)):
    test_text[i]['label'] = test_label[i]

In [14]:
def tokenize_data(data):
    tokenized = tokenizer(data['text'], padding='max_length', truncation=True, max_length=40)
    tokenized['label'] = data['label']
    return tokenized
tokenized_data = [tokenize_data(d) for d in train_text]
test_tokenized_data = [tokenize_data(d) for d in test_text]

In [15]:
class DocumentClassificationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        d = self.data[index]
        input_ids = torch.tensor(d['input_ids'])
        attention_mask = torch.tensor(d['attention_mask'])
        label = torch.tensor(d['label'])
        item = {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': label}
        return item

dataset = DocumentClassificationDataset(tokenized_data)
test_dataset = DocumentClassificationDataset(test_tokenized_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

test_dataloader = DataLoader(test_tokenized_data, batch_size=8, shuffle=True)

In [16]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,   # batch size per device during training
    per_device_eval_batch_size=1,    # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=10,
    save_steps=10,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to = 'wandb'
)

In [17]:
trainer = Trainer(
    model=model,                     # the instantiated Transformers model to be trained
    args=training_args,              # training arguments, defined above
    train_dataset=dataset, 
    eval_dataset = test_dataset, # training dataset
    data_collator=lambda data: {'input_ids': torch.stack([f['input_ids'] for f in data]),
                                'attention_mask': torch.stack([f['attention_mask'] for f in data]),
                                'labels': torch.stack([f['label'] for f in data])},
    # data collator is required when using batched datasets
)

In [18]:
start = time.time()

In [19]:
trainer.train()
print('training time is',time.time() - start)

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: guoydora (yueguo). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,1.399800,1.389495
20,1.396100,1.383197
30,1.375500,1.371683
40,1.361100,1.341222
50,1.289800,1.197915
60,1.008800,0.679045
70,0.590800,0.424503
80,0.452700,0.405557
90,0.377200,0.420430
100,0.455400,0.376360


/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.w

training time is 4964.653178453445


In [ ]:
import matplotlib.pyplot as plt

train_loss = trainer.history['loss']
plt.plot(train_loss)
plt.xlabel('Batch')
plt.ylabel('Training Loss')
plt.show()

In [ ]:
for i,j in tokenized_data[0].items():
    print(i)

In [ ]:
tokenized_data[0]

In [ ]:
set(train_label)